In [ ]:
from sqlalchemy import create_engine, inspect, func

In [ ]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

# Use Inspector to print the column names and types
columns = inspector.get_columns('emoji')
for c in columns:
    print(c['name'], c["type"])

# Use `engine.execute` to select and display the first 10 rows from the emoji table
engine.execute('SELECT * FROM emoji LIMIT 10').fetchall()

# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Emoji = Base.classes.emoji

# Start a session to query the database
session = Session(engine)

# Query Emojis for `emoji_char`, `emoji_id`, and `score` and save the query into results
results = session.query(Emoji.emoji_char, Emoji.emoji_id, Emoji.score).\
    order_by(Emoji.score.desc()).all()
print(results)

# Unpack the `emoji_id` and `scores` from results and save into separate lists
emoji_id = [result[1] for result in results[:10]]
scores = [int(result[2]) for result in results[:10]]

# Load the results into a pandas dataframe. Set the index to the `emoji_id`
df = pd.DataFrame(results[:10], columns=['emoji_char', 'emoji_id', 'score'])
df.set_index('emoji_id', inplace=True, )
df.head(10)

# BONUS: Use Pandas `read_sql_query` to load a query statement directly into the DataFrame
stmt = session.query(Emoji).\
    order_by(Emoji.score.desc()).statement
df2 = pd.read_sql_query(stmt, session.bind)
df2.head(10)

In [ ]:
# Collect the names of tables within the database
inspector.get_table_names()

# Using the inspector to print the column names within the 'Salaries' table and its types
columns = inspector.get_columns('Salaries')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

# Assign the dow class to a variable called `Dow`
Dow = Base.classes.dow

# Create a session
session = Session(engine)

# Display the row's columns and data in dictionary format
first_row = session.query(Dow).first()
first_row.__dict__

# Use the session to query Dow table and display the first 5 trade volumes
for row in session.query(Dow.stock, Dow.volume).limit(15).all():
    print(row)

In [ ]:
# Note that adding to the session does not update the table. It queues up those queries.
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

# The data hasn't been added yet
engine.execute('select * from pet').fetchall()

# We can use the new attribute to see the queue of data ready to go into the database
session.new

# commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()

# Nothing new to add
session.new

# query the database
session.query(Pet.name, Pet.type, Pet.age).all()

# Create a query and then run update on it
# use first() to get a single result - what if we didn't?
pet = session.query(Pet).filter_by(name="Marshmallow").first()
pet.age += 1

# For modifications, we can use the dirty attribute
session.dirty

# Commit Transaction
session.commit()

# Session is up-to-date
session.dirty

session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

# Create a query and then delete the row collected
pet = session.query(Pet).filter_by(id=4).delete()
session.commit()

session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()


In [ ]:
# Print all of the player names in the database
players = session.query(BaseballPlayer)
for player in players:
  print(player.name_given)

# Find the number of players from the USA
usa = session.query(BaseballPlayer).\
    filter(BaseballPlayer.birth_country == 'USA').count()
print("There are {} players from the USA".format(usa))

# Same thing, different syntax
usa = (session.query(BaseballPlayer).
    filter(BaseballPlayer.birth_country == 'USA').count())
print("There are {} players from the USA".format(usa))

# Same thing, but using filter_by
# https://stackoverflow.com/questions/2128505/whats-the-difference-between-filter-and-filter-by-in-sqlalchemy
usa = (session.query(BaseballPlayer).
    filter_by(birth_country = 'USA').count())
print("There are {} players from the USA".format(usa))

# Find those players who were born before 1990
born_before_1990 = session.query(BaseballPlayer).\
    filter(BaseballPlayer.birth_year < 1990).count()
    
print("{} players were born before 1990".format(born_before_1990))

# Find those players from the USA who were born after 1989
born_after_1989 = (session.query(BaseballPlayer).
    filter(BaseballPlayer.birth_year > 1989).filter(BaseballPlayer.birth_country == "USA").
    count())
print("{} USA players were born after 1989".format(born_after_1989))

In [ ]:
# Query the database and collect all of the surfers in the Surfer table
surfer_list = session.query(Surfer)
for bro in surfer_list:
    print(bro.name)
    print(bro.hometown)
    print(bro.rank)